# Lab 3: Alignment-based metrics in Machine Learning

* Author: Romain Tavenard (@rtavenar)
* License: CC-BY-NC-SA

A lab session from a course on Machine Learning for Time Series at ENSAI.
One can find lecture notes for this course [there](https://rtavenar.github.io/ml4ts_ensai/).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tslearn.metrics import dtw, soft_dtw
from tslearn.barycenters import dtw_barycenter_averaging, softdtw_barycenter

# Data loading

**Question #1.** Using the [`CachedDatasets`](https://tslearn.readthedocs.io/en/stable/gen_modules/datasets/tslearn.datasets.CachedDatasets.html#tslearn.datasets.CachedDatasets)
utility from ``tslearn``, load the "Trace" time series dataset.
What are the dimensions of an array storing a time series dataset?
Create a new dataset `X_subset` made of 50 random time series from classes indexed 1 to 3 (`y_train < 4`)
in the training set.

# $k$-means clustering

**Question #2.** Implement the Lloyd's algorithm for a $k$-means that would use soft-DTW as 
its base metric.
You can rely on ``tslearn`` functions (see imports above) for "distance" computations and barycenter
estimation.
Your function should return both the current assignments and the barycenters.
Check that it runs smoothly for a few iterations on `X_subset` (leave quantitative evaluation aside for now).

In [ ]:
def kmeans_soft_dtw(X, gamma, k, max_iter=10):
    n_timeseries, n_timestamps, n_features = X.shape

    # Init barycenters at random
    barycenters = X[np.random.randint(n_timeseries, size=k)]

    for e in range(max_iter):
        # Assign a cluster to each time series
        assign = np.zeros((n_timeseries, ), dtype=np.int)
        # for i in range(n_timeseries):
            # TODO
            # assign[i] = ...
        # Update centroids (barycenters) for each cluster
        # for j in range(k):
            # TODO
            # barycenters[j] = ...
    
    return assign, barycenters



**Question #3.** Implement the Lloyd's algorithm for a $k$-means that would use **DTW** as 
its base metric.
You can rely on ``tslearn`` functions (see imports above) for "distance" computations and barycenter
estimation.
Your function should return both the current assignments and the barycenters.
Check that it runs smoothly for a few iteration on `X_subset` (leave quantitative evaluation aside for now).

In [ ]:
def kmeans_soft_dtw(X, k, max_iter=10):
    n_timeseries, n_timestamps, n_features = X.shape

    # Init barycenters at random
    barycenters = X[np.random.randint(n_timeseries, size=k)]

    for e in range(max_iter):
        # Assign a cluster to each time series
        assign = np.zeros((n_timeseries, ), dtype=np.int)
        # for i in range(n_timeseries):
            # TODO
            # assign[i] = ...
        # Update centroids (barycenters) for each cluster
        # for j in range(k):
            # TODO
            # barycenters[j] = ...
    
    return assign, barycenters

**Question #4.** Implement a function that would assess the quality of a clustering in terms of 
intra-cluster inertia, computed using **DTW** as the base metric.
Your function should take a time series dataset, corresponding assignments and barycenters as inputs.

In [ ]:
def dtw_cost(X, assign, barycenters):
    n_timeseries, n_timestamps, n_features = X.shape

    total_cost = 0.
    # TODO
    
    return total_cost


**Question #5.** Compare your $k$-means implementations in terms of DTW inertia.
For a fair comparison, make sure that they are initialized similarly, by appropriately setting your
random number generator seeds.
What do you observe? Is that expected / Do you have an explanation for that?

# Multi-step ahead forecasting

In this section, your goal will be to implement a single-hidden-layer perceptron for time series forecasting.
Your network will be trained to minimize normalized soft-DTW[^1].

To do so, we will rely on a (very nice!) `torch`-compatible implementation of soft-DTW obtained from 
[github](https://github.com/Maghoumi/pytorch-softdtw-cuda) 
(the corresponding file is already added to this project).

The code below is an implementation of a generic Multi-Layer-Perceptron class in `torch`, and you will rely on it for your implementation of a forecasting MLP with softDTW loss.

[^1]: Normalized soft-DTW (also coined soft-DTW divergence) between time series $\mathbf{x}$ and 
$\mathbf{x}^\prime$ is defined as: 
$$\text{soft-DTW}(\mathbf{x}, \mathbf{x}^\prime) - \frac{1}{2} \left( \text{soft-DTW}(\mathbf{x}, \mathbf{x}) + \text{soft-DTW}(\mathbf{x}^\prime, \mathbf{x}^\prime) \right)$$


In [ ]:
import torch
from torch import nn

class MultiLayerPerceptron(torch.nn.Module):  # No hidden layer here
    def __init__(self, layers, loss=None):
        # At init, we define our layers
        super(MultiLayerPerceptron, self).__init__()
        self.layers = layers
        if loss is None:
            self.loss = torch.nn.MSELoss(reduction="none")
        else:
            self.loss = loss
        self.optimizer = torch.optim.SGD(self.parameters(), lr=0.001)
            

    def forward(self, X):
        # The forward method informs about the forward pass: how one computes outputs of the network
        # from the input and the parameters of the layers registered at init
        if not isinstance(X, torch.Tensor):
            X = torch.Tensor(X)
        batch_size = X.size(0)
        X_reshaped = torch.reshape(X, (batch_size, -1))  # Manipulations to deal with time series format
        output = self.layers(X_reshaped)
        return torch.reshape(output, (batch_size, -1, 1))  # Manipulations to deal with time series format
    
    def fit(self, X, y, max_epochs=10):
        # The fit method performs the actual optimization
        X_torch = torch.Tensor(X)
        y_torch = torch.Tensor(y)

        for e in range(max_epochs):
            self.optimizer.zero_grad()
            # Forward pass
            y_pred = self.forward(X_torch)
            # Compute Loss
            loss = self.loss(y_pred, y_torch).mean()
            if e % 20 == 0:
                print('Epoch {}: train loss: {}'.format(e, loss.item()))
            # Backward pass
            loss.backward()
            self.optimizer.step()

# Example definition of a MLP model using the class above
model = MultiLayerPerceptron(
    layers=nn.Sequential(
        nn.Linear(in_features=150, out_features=256),
        nn.ReLU(),
        nn.Linear(in_features=256, out_features=3),
        nn.ReLU()
    )
)
# model.fit(X, y, max_epochs=100)  # Here one needs to define what X and y are, obviously

**Question #6.** Take inspiration from the code above to define an MLP class that would allow training
a single-hidden-layer model using normalized soft-DTW as a criterion to be optimized.
Train your network for 200 epochs on a forecasting task that would consist, given the first 150 elements
of a time series, in predicting the next 125 ones.

In [ ]:
from soft_dtw_cuda import SoftDTW




**Question #7.** Use the following code block to qualitatively assess quality of your 
network's predictions.
Do not hesitate to change the time series index to visualize the result for different time series.

In [ ]:
import matplotlib.pyplot as plt

ts_index = 50

y_pred = model(X_test[:, :150, 0]).detach().numpy()

plt.figure()
plt.plot(X_test[ts_index].ravel())
plt.plot(np.arange(150, 275), y_pred[ts_index], 'r-')
plt.show()

# Shapelet models

Shapelet models are time series classifiers that rely on the presence/absence of local shapes in 
time series to make predictions.
These models will be presented in more details later in the course, since they can be seen as a variant of 
shallow convolutional models for time series.

In the following, you will train such models (using their 
[`tslearn` implementation](https://tslearn.readthedocs.io/en/stable/gen_modules/shapelets/tslearn.shapelets.LearningShapelets.html#tslearn.shapelets.LearningShapelets)) 
and visualize their learned decision boundaries.
For the sake of visualization, models made of only 2 local shapes will be learned, and the following 
function will be used to visualize them:

In [ ]:
from matplotlib import cm

def visualize_shapelet_model_2_shapelets(model, X, y):
    distances = model.transform(X)

    plt.figure()
    viridis = cm.get_cmap('viridis', 4)
    # Create a scatter plot of the 2D distances for the time series of each class.
    for i, yi in enumerate(np.unique(y)):
        plt.scatter(distances[y == yi][:, 0],
                    distances[y == yi][:, 1],
                    color=viridis(i / 3),
                    edgecolors='k',
                    label='Class {}'.format(yi))

    # Create a meshgrid of the decision boundaries
    xmin = np.min(distances[:, 0]) - 0.1
    xmax = np.max(distances[:, 0]) + 0.1
    ymin = np.min(distances[:, 1]) - 0.1
    ymax = np.max(distances[:, 1]) + 0.1
    xx, yy = np.meshgrid(np.arange(xmin, xmax, (xmax - xmin) / 200), 
                         np.arange(ymin, ymax, (ymax - ymin) / 200))
    
    weights, biases = model.get_weights('classification')
    Z = []
    for x, y in np.c_[xx.ravel(), yy.ravel()]:
        Z.append(np.argmax([biases[i] + weights[0][i]*x + weights[1][i]*y
                            for i in range(4)]))
    Z = np.array(Z).reshape(xx.shape)
    plt.contourf(xx, yy, Z / 3, cmap=viridis, alpha=0.25)

    plt.legend()
    plt.xlabel('$d(\mathbf{x}, \mathbf{s}_1)$')
    plt.ylabel('$d(\mathbf{x}, \mathbf{s}_2)$')
    plt.xlim((xmin, xmax))
    plt.ylim((ymin, ymax))
    plt.title('Distance transformed time series')
    plt.show()

Let us start with a first attempt.
The following code snippet defines a model made of 2 shapelets (=local shapes to be learnt) and train
them (as well as the corresponding linear decision boundaries) for 500 epochs:

In [ ]:
from tslearn.shapelets import LearningShapelets

shp_clf = LearningShapelets(
    n_shapelets_per_size={20: 2},  # 2 shapelets of length 20
    max_iter=500,
    verbose=0,                     # Do not print information on the evolution of the loss
    scale=False,
    random_state=42
)
shp_clf.fit(X_train, y_train)

**Question #8.** Visualize the obtained decision boundaries. 
Would you expect this model to be a decent classifier?

**Question #9.** Shapelet models usually benefit from scaling of the input time series 
(yet this is, of course, very dataset-specific).
See if this helps in our case.

**Question #10.** By default, the `tslearn` implementation uses SGD as an optimizer.
See if using `"adam"` helps in our case.

**Question #11.** Set the learning rate of the Adam optimizer to 1e-2 
(you will need to import the `Adam` class from `keras` as follows) and see if this improves the learning.

In [ ]:
from tensorflow.keras.optimizers import Adam

